# Paper Replication: de Cheveigné (2020)

**"ZapLine: A simple and effective method to remove power line artifacts"** - *NeuroImage*

This notebook replicates key results from the ZapLine paper.

## Key Results to Replicate
- PSD before/after ZapLine
- Comparison with notch filter
- Signal preservation at other frequencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from mne_denoise.dss import dss_zapline, compute_psd_reduction

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

## 1. Generate Realistic EEG with Line Noise

Simulate multichannel EEG with:
- Background 1/f noise
- Alpha rhythm (10 Hz)
- Strong 50 Hz line noise

In [ ]:
# Parameters
sfreq = 500
duration = 60  # seconds
n_samples = int(duration * sfreq)
n_channels = 64
t = np.arange(n_samples) / sfreq

# 1. Generate 1/f background noise
def generate_pink_noise(n_samples, n_channels):
    """Generate pink (1/f) noise."""
    freqs = np.fft.rfftfreq(n_samples, 1)
    freqs[0] = 1e-6  # Avoid division by zero
    pink_filter = 1 / np.sqrt(freqs)
    
    noise = np.zeros((n_channels, n_samples))
    for ch in range(n_channels):
        white = np.random.randn(n_samples)
        spectrum = np.fft.rfft(white)
        pink_spectrum = spectrum * pink_filter
        noise[ch] = np.fft.irfft(pink_spectrum, n_samples)
    return noise

eeg = generate_pink_noise(n_samples, n_channels) * 20  # Scale to ~20 µV

# 2. Add alpha rhythm (10 Hz) with spatial pattern
alpha_mixing = np.random.randn(n_channels)
alpha_mixing[30:50] *= 3  # Stronger in posterior channels
alpha_mixing /= np.linalg.norm(alpha_mixing)
alpha = np.sin(2 * np.pi * 10 * t) * (1 + 0.5 * np.sin(2 * np.pi * 0.2 * t))
eeg += 15 * np.outer(alpha_mixing, alpha)

# 3. Add 50 Hz line noise (strong, correlated across channels)
line_mixing = np.ones(n_channels) + 0.2 * np.random.randn(n_channels)
line_mixing /= np.linalg.norm(line_mixing)
line_noise = np.sin(2 * np.pi * 50 * t)
eeg_noisy = eeg + 30 * np.outer(line_mixing, line_noise)

print(f"Data: {n_channels} channels, {duration} seconds")
print(f"Line noise amplitude: ~30 µV")

## 2. Apply ZapLine

In [ ]:
# Apply ZapLine
result = dss_zapline(
    eeg_noisy,
    line_freq=50,
    sfreq=sfreq,
    n_remove='auto'
)

eeg_clean = result.cleaned

print(f"Components removed: {result.n_removed}")

# Compute reduction
metrics = compute_psd_reduction(eeg_noisy, eeg_clean, sfreq, 50)
print(f"50 Hz reduction: {metrics['reduction_db']:.1f} dB")

## Figure: PSD Before/After ZapLine

Key figure from the paper showing line noise removal.

In [ ]:
def compute_avg_psd(data, sfreq):
    """Compute average PSD across channels."""
    nperseg = int(4 * sfreq)
    freqs, psd = signal.welch(data, sfreq, nperseg=nperseg, axis=1)
    return freqs, psd.mean(axis=0)

freqs, psd_noisy = compute_avg_psd(eeg_noisy, sfreq)
freqs, psd_clean = compute_avg_psd(eeg_clean, sfreq)
freqs, psd_orig = compute_avg_psd(eeg, sfreq)  # Without line noise

# Main figure
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# (a) Full spectrum
ax = axes[0]
ax.semilogy(freqs, psd_noisy, 'r-', alpha=0.7, label='Before ZapLine')
ax.semilogy(freqs, psd_clean, 'b-', alpha=0.7, label='After ZapLine')
ax.semilogy(freqs, psd_orig, 'g--', alpha=0.5, label='True (no line noise)')
ax.axvline(50, color='red', linestyle=':', alpha=0.5)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('PSD (µV²/Hz)')
ax.set_title('(a) Power Spectral Density')
ax.set_xlim([0, 100])
ax.legend()

# (b) Zoom on 50 Hz
ax = axes[1]
mask = (freqs >= 45) & (freqs <= 55)
ax.semilogy(freqs[mask], psd_noisy[mask], 'r-', linewidth=2, label='Before')
ax.semilogy(freqs[mask], psd_clean[mask], 'b-', linewidth=2, label='After')
ax.semilogy(freqs[mask], psd_orig[mask], 'g--', linewidth=2, label='True')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('PSD')
ax.set_title(f'(b) Zoom at 50 Hz\n({metrics["reduction_db"]:.1f} dB reduction)')
ax.legend()

# (c) Alpha band preservation
ax = axes[2]
mask = (freqs >= 5) & (freqs <= 20)
ax.plot(freqs[mask], psd_noisy[mask], 'r-', linewidth=2, label='Before')
ax.plot(freqs[mask], psd_clean[mask], 'b-', linewidth=2, label='After')
ax.plot(freqs[mask], psd_orig[mask], 'g--', linewidth=2, label='True')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('PSD')
ax.set_title('(c) Alpha band (8-12 Hz) preserved')
ax.legend()

plt.suptitle('ZapLine: Power Line Artifact Removal\n(de Cheveigné 2020)', fontsize=14)
plt.tight_layout()
plt.savefig('paper3_zapline_psd.png', dpi=150)
plt.show()

## Comparison with Notch Filter

Paper's key point: ZapLine removes only the spatial artifact, not the entire frequency band.

In [ ]:
# Apply traditional notch filter
from scipy.signal import iirnotch, filtfilt

b, a = iirnotch(50, Q=30, fs=sfreq)
eeg_notch = filtfilt(b, a, eeg_noisy, axis=1)

freqs, psd_notch = compute_avg_psd(eeg_notch, sfreq)

# Compare
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# (a) PSD comparison
ax = axes[0]
mask = (freqs >= 40) & (freqs <= 60)
ax.semilogy(freqs[mask], psd_noisy[mask], 'r-', linewidth=2, label='Original')
ax.semilogy(freqs[mask], psd_notch[mask], 'm-', linewidth=2, label='Notch filter')
ax.semilogy(freqs[mask], psd_clean[mask], 'b-', linewidth=2, label='ZapLine')
ax.semilogy(freqs[mask], psd_orig[mask], 'g--', linewidth=1, label='True')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('PSD')
ax.set_title('(a) Notch vs ZapLine')
ax.legend()

# (b) Time domain at 50 Hz channel
ax = axes[1]
t_plot = t[:1000]  # 2 seconds
ch = 0
ax.plot(t_plot, eeg_noisy[ch, :1000], 'r-', alpha=0.5, label='Original')
ax.plot(t_plot, eeg_notch[ch, :1000], 'm-', alpha=0.7, label='Notch')
ax.plot(t_plot, eeg_clean[ch, :1000], 'b-', alpha=0.7, label='ZapLine')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude (µV)')
ax.set_title('(b) Time domain (Channel 1)')
ax.legend()

plt.suptitle('ZapLine vs Notch Filter Comparison', fontsize=14)
plt.tight_layout()
plt.savefig('paper3_notch_comparison.png', dpi=150)
plt.show()

# Quantitative comparison
metrics_notch = compute_psd_reduction(eeg_noisy, eeg_notch, sfreq, 50)
print(f"\n50 Hz reduction:")
print(f"  ZapLine: {metrics['reduction_db']:.1f} dB")
print(f"  Notch filter: {metrics_notch['reduction_db']:.1f} dB")

## Harmonics Removal (100 Hz, 150 Hz)

In [ ]:
# Add harmonics
eeg_harmonics = eeg_noisy.copy()
eeg_harmonics += 10 * np.outer(line_mixing, np.sin(2 * np.pi * 100 * t))  # 2nd harmonic
eeg_harmonics += 5 * np.outer(line_mixing, np.sin(2 * np.pi * 150 * t))   # 3rd harmonic

# ZapLine with harmonics
result_harm = dss_zapline(
    eeg_harmonics,
    line_freq=50,
    sfreq=sfreq,
    n_harmonics=3,
    n_remove='auto'
)

# Check all harmonics
freqs_harm, psd_harm_before = compute_avg_psd(eeg_harmonics, sfreq)
freqs_harm, psd_harm_after = compute_avg_psd(result_harm.cleaned, sfreq)

fig, ax = plt.subplots(figsize=(12, 4))
ax.semilogy(freqs_harm, psd_harm_before, 'r-', label='Before')
ax.semilogy(freqs_harm, psd_harm_after, 'b-', label='After ZapLine')
for h in [50, 100, 150]:
    ax.axvline(h, color='gray', linestyle=':', alpha=0.5)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('PSD')
ax.set_title('ZapLine with Harmonics (50, 100, 150 Hz)')
ax.set_xlim([0, 200])
ax.legend()
plt.tight_layout()
plt.savefig('paper3_harmonics.png', dpi=150)
plt.show()

# Reduction at each harmonic
for freq in [50, 100, 150]:
    m = compute_psd_reduction(eeg_harmonics, result_harm.cleaned, sfreq, freq)
    print(f"{freq} Hz reduction: {m['reduction_db']:.1f} dB")

## Summary

Replicated key results from de Cheveigné (2020):

1. **>20 dB reduction** at line frequency
2. **Preserves brain signals** at other frequencies (alpha 10 Hz)
3. **Better than notch filter** - removes spatial artifact, not entire frequency band
4. **Handles harmonics** (100, 150 Hz)

In [ ]:
print("=" * 50)
print("REPLICATION SUMMARY")
print("=" * 50)
print(f"\n50 Hz reduction: {metrics['reduction_db']:.1f} dB")
print(f"Components removed: {result.n_removed}")

# Check alpha preservation
alpha_before = compute_psd_reduction(eeg, eeg_noisy, sfreq, 10, bandwidth=4)
alpha_after = compute_psd_reduction(eeg, eeg_clean, sfreq, 10, bandwidth=4)
print(f"\n10 Hz alpha preservation: {alpha_after['power_cleaned']/alpha_before['power_cleaned']*100:.1f}%")

print("\n" + "=" * 50)
print("Saved figures: paper3_zapline_psd.png, paper3_notch_comparison.png, paper3_harmonics.png")